In [ ]:
import numpy as np 
import pandas as pd
import sklearn 
import tensorflow as tf 
import nltk
import gensim 

### Load Data

In [ ]:
##TO-DO: Read dataset.json as a dictionary object 

In [ ]:
##TO-DO: Convert dictionary object to pandas dataframe 

### Pre-Process Data

In [ ]:
def clean_text(text_features):
    """
    pre-process text features of our dataset:  
    - Case Lowering
    - Tokenization
    - Stop Word Removal
    - Any other you want, including TF-IDF filtering 
 
    Paramaters
    ---------
    text_features: text (tweet) data in our dataframe.
    """

    ##TO-DO: complete this function!
    ##When determining how to return the processed data, consider the format your data needs to be in for train_test_split and model training

 

In [ ]:
def vectorize_text(cleaned_text, use_word2vec=True):
    """
    use Word2Vec or Sentence2Vec to vectorize the cleaned text data
 
    Paramaters
    ---------
    cleaned_text: (cleaned) text (tweet) data in our dataframe.
    """
    ##TO-DO: complete this function!
    ##When determining how to return the processed data, consider the format your data needs to be in for train_test_split and model training


### Model Training: KNN/Random Forest/RNNs

In [ ]:
from sklearn.model_selection import train_test_split

## TO-DO: split our data into train and test data

##### Using KNN
###### Refer to this resource: https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification

In [ ]:
## TO-DO: Determine initial value of parameters (especially K)
k = 0

## TO-DO: determine the best distance computation metric
candidate_metrics = ['euclidean','cosine','manhattan']
best_metric = ''




In [ ]:
from sklearn.neighbors import KNeighborsClassifier

## TO-DO: fit a KNN Classifier with distance metric and K 

In [ ]:
# TO-DO: calculating accuracy
from sklearn.metrics import accuracy_score


In [ ]:
## TO-DO: optimize K with hyperparameter tuning

from sklearn.model_selection import cross_val_score


## TO-DO: Visualize results to determine best K


In [ ]:
## TO-DO: repeat training with optimized K

##### Using XGBoost/Random Forest
###### Refer to this resource: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
## TO-DO: Train model naively, with default parameters
from sklearn.ensemble import RandomForestClassifier

In [ ]:
## TO-DO: Evaluate base model
from sklearn.metrics import accuracy_score



In [ ]:
##TO-DO: Hyperparameter Tuning via RandomizedSearch & GridSearch 

## get current parameters and their values 
from pprint import pprint


## Randomized Search 
from sklearn.model_selection import RandomizedSearchCV
random_grid = {}
rf_random = RandomizedSearchCV()

## Grid Search
from sklearn.model_selection import GridSearchCV
param_grid = {}
grid_search = GridSearchCV()

##### Using LSTMs
###### Refer to this resource: https://www.youtube.com/watch?v=8HyCNIVRbSU&ab_channel=TheA.I.Hacker-MichaelPhi

In [ ]:
from tensorflow.python.keras.layers import LSTM, Activation, Dropout, Dense, Embedding
from tensorflow.python.keras.models import Sequential
from keras_preprocessing.sequence import pad_sequences


In [ ]:
## veryyyyyy basic architecture. TO-DO: complete and expand

model = Sequential()
model.add(Embedding())
model.add(LSTM())
model.add(Dense(2, activation='softmax'))

model.compile(loss='',optimizer='', metrics=['accuracy'])

In [ ]:
epochs = 0
batch_size = 0

## TO-DO: determine training parameters and complete call
model.fit()

In [ ]:
## TO-DO: evaluate function 
model.evaluate()

### Using BERT
###### Refer to: http://jalammar.github.io/illustrated-bert/ 
###### Refer to: https://medium.com/@parthdholakiya180/twitter-hate-detection-using-bert-e7682b2d0a0c 

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer